In [139]:
import os
import pandas as pd
from datetime import datetime, timedelta


In [140]:
def get_user_list(data_path):
    """
    Returns a list of unique user IDs from the Excel file.

    Args:
        data_path (str): Path to the Excel file.

    Returns:
        list or None: List of unique user IDs, or None if an error occurs.
    """
    try:
        df = pd.read_excel(data_path)
    except FileNotFoundError:
        print(f"Error: File not found: {data_path}")
        return None
    except pd.errors.ParserError:
        print(f"Error: Could not parse Excel file: {data_path}. Check if it's a valid Excel format.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while reading the Excel file: {e}")
        return None

    try:
        users = df["userId"].unique().tolist() # convert numpy array to list
        return users
    except KeyError:
        print("Error: 'userId' column not found in the Excel file.")
        return None
    except Exception as e:
        print(f"An error occurred while getting user list: {e}")
        return None

In [141]:
def filter_user_data(data_path, target_user=None, num_days=10):
    """
    Filters search query data for a specific user within a given number of days.

    Args:
        data_path (str): Path to the Excel file containing search queries.
        target_user: User ID to filter for. If None, the first user in the data is used.
        num_days (int): Number of most recent days to filter data for. Defaults to 10.

    Returns:
        pandas.DataFrame or None: Filtered DataFrame if successful, None otherwise.
        Prints informative messages to the console in case of errors or info.
    """
    try:
        df = pd.read_excel(data_path)
    except FileNotFoundError:
        print(f"Error: File not found: {data_path}")
        return None
    except pd.errors.ParserError:
        print(f"Error: Could not parse Excel file: {data_path}. Check if it's a valid Excel format.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while reading the Excel file: {e}")
        return None

    print(f"Total number of records: {len(df)}")

    try:
        df["search_date"] = pd.to_datetime(df["search_date"])
    except KeyError:
        print("Error: 'search_date' column not found in the Excel file.")
        return None
    except Exception as e:
        print(f"An error occurred while converting 'search_date' to datetime: {e}")
        return None

    try:
        df["query"] = df["query"].astype(str)
    except KeyError:
        print("Error: 'query' column not found in the Excel file.")
        return None
    except Exception as e:
        print(f"An error occurred while converting 'query' to string: {e}")
        return None

    total_dates = df["search_date"].unique()
    print(f"Number of unique search dates: {len(total_dates)}")

    users = df["userId"].unique()
    print(f"Number of users: {len(users)}")

    if not users.size:
        print("No users found in the data.")
        return None

    if target_user is None:
        target_user = users[0]
        print(f"No target user provided. Filtering data for first user: {target_user}")
    elif target_user not in users:
        print(f"Error: User {target_user} not found in the data.")
        return None
    else:
        print(f"Filtering data for user: {target_user}")

    filtered_user_data = df[df["userId"] == target_user]
    print(f"Number of records for user {target_user}: {len(filtered_user_data)}")

    if len(total_dates) < num_days:
        num_days = len(total_dates)
        print(f"Note: Less than {num_days} unique dates found. Filtering for all available dates.")

    top_n_dates = total_dates[:num_days]

    filtered_ten_days_data = filtered_user_data[filtered_user_data["search_date"].isin(top_n_dates)]
    print(f"Number of records for user {target_user} within the last {num_days} days: {len(filtered_ten_days_data)}")
    return filtered_ten_days_data


In [142]:
def get_comma_separated_queries(df):
    """
    Returns a comma-separated string of queries from a DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame containing a 'query' column.

    Returns:
        str or None: Comma-separated string of queries, or None if the DataFrame is empty or 'query' column is missing.
    """
    if df is None or df.empty:
        print("Warning: Input DataFrame is empty.")
        return None
    if "query" not in df.columns:
        print("Warning: 'query' column not found in the DataFrame.")
        return None
    
    queries = df["query"].tolist()
    # Remove any None values and empty strings before joining
    queries = [q for q in queries if q is not None and str(q).strip() != ""]
    if not queries: #check if the list is empty after removing None values and empty strings.
        print("Warning: No valid queries found in the DataFrame.")
        return None
    comma_separated_queries = ", ".join(queries)
    return comma_separated_queries


In [ ]:
data_path = os.path.join(os.getcwd(), "search_output", "search_queries.xlsx")

user_list = get_user_list(data_path)

if user_list:
    print("\nList of users:", user_list)
    
    # Pass the first user from the list to filter_user_data
    first_user = user_list[0]
    filtered_data = filter_user_data(data_path, target_user=first_user)

    if filtered_data is not None:
        queries_string = get_comma_separated_queries(filtered_data)
        if queries_string:
            print(f"\nComma-separated queries for user {first_user}:")
            print(queries_string)
else:
    print("Could not retrieve user list.")